# Object Detection with MPA
To test this sample, 'data' folder should be linked to the location of this file.

## Import everything

In [ ]:
import os
import sys

import cv2
import numpy as np

from ote_sdk.configuration.helper import create as create_parameters_from_parameters_schema
from ote_sdk.entities.inference_parameters import InferenceParameters
from ote_sdk.entities.label_schema import LabelSchemaEntity
from ote_sdk.entities.model import ModelEntity, ModelOptimizationType, ModelPrecision, ModelStatus
from ote_sdk.entities.model_template import parse_model_template, TargetDevice
from ote_sdk.entities.resultset import ResultSetEntity
from ote_sdk.entities.subset import Subset
from ote_sdk.entities.task_environment import TaskEnvironment
from ote_sdk.usecases.tasks.interfaces.export_interface import ExportType

from ote_cli.datasets import get_dataset_class
from ote_cli.registry import Registry
from ote_cli.utils.importing import get_impl_class

from mpa import MPAConstants


print(f'pkg root = {MPAConstants.PACKAGE_ROOT}')
print(f'recipes = {MPAConstants.RECIPES_PATH}')
print(f'cwd = {os.getcwd()}')

assert os.path.exists('./data')


## Register templates

In [ ]:
templates_dir = '../../models/templates'
registry = Registry(templates_dir)
# registry = registry.filter(task_type=sys.executable.split(os.sep)[-4])
print(registry)

## Load model template and its hyper parameters

In [ ]:
model_template = registry.get('Advanced_Transfer_Learning_for_VFNet_object_detection')
hyper_parameters = model_template.hyper_parameters.data

## Get dataset instantiated

In [ ]:
Dataset = get_dataset_class(model_template.task_type)

# dataset_paths = {
#     # 'train_ann_file' : './data/coco/annotations/semi_supervised/instances_train2017.1@0.1.json',
#     # 'train_data_root' : './data/coco/annotations/train2017/',
#     # 'val_ann_file' : './data/coco/annotations/semi_supervised/instances_val2017.1@0.1.json',
#     # 'val_data_root' : './data/coco/annotations/val2017/',
#     'train_ann_file' : './external/training_extensions/data/airport/annotation_faces_train.json',
#     'train_data_root' : './external/training_extensions/data/airport/',
#     'val_ann_file' : './external/training_extensions/data/airport/annotation_faces_train.json',
#     'val_data_root' : './external/training_extensions/data/airport/',
# }

# dataset = Dataset(**dataset_paths)
# labels_schema = LabelSchemaEntity.from_labels(dataset.get_labels())
dataset = Dataset(
    train_subset={'ann_file': '../../../../data/airport/annotation_faces_train.json',
                  'data_root':  '../../../../data/airport/'},
    val_subset={'ann_file': '../../../../data/airport/annotation_faces_train.json',
                'data_root': '../../../../data/airport'}
)
labels_schema = LabelSchemaEntity.from_labels(dataset.get_labels())

# print(f'validation dataset = {dataset.get_subset(Subset.VALIDATION)}')
# for i, ditem in enumerate(dataset.get_subset(Subset.VALIDATION), 1):
#     print(f'dataset item {i} = {ditem.numpy}')

## Have a look at existing parameters

In [ ]:
hyper_parameters = create_parameters_from_parameters_schema(hyper_parameters)

for p in hyper_parameters.learning_parameters.parameters:
    print(f'{p}: {getattr(hyper_parameters.learning_parameters, p)}')

## Tweak parameters

In [ ]:
hyper_parameters.learning_parameters.batch_size = 16
hyper_parameters.learning_parameters.num_iters = 150

for p in hyper_parameters.learning_parameters.parameters:
    print(f'{p}: {getattr(hyper_parameters.learning_parameters, p)}')

## Create Task 

In [ ]:
Task = get_impl_class(model_template.entrypoints.base)

environment = TaskEnvironment(
    model=None,
    hyper_parameters=hyper_parameters,
    label_schema=labels_schema,
    model_template=model_template)
        
task = Task(task_environment=environment)

## Run training

In [8]:
output_model = ModelEntity(
    dataset,
    environment.get_model_configuration(),
    model_status=ModelStatus.NOT_READY)

task.train(dataset, output_model)

2022-01-26 13:54:57,044 | INFO : Saving checkpoint at 2 epochs
2022-01-26 13:54:57,046 | INFO : ----------------- UnbiasedTeacher.state_dict_hook() called
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 5/5, 21.4 task/s, elapsed: 0s, ETA:     0s2022-01-26 13:54:57,733 | INFO : Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.03s).
Accumulating evaluation results...
DONE (t=0.01s).
Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.002
Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.006
Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.000
Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.000
Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.000
Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.006
Average Recall     (AR)

## Evaluate quality metric

In [9]:
validation_dataset = dataset.get_subset(Subset.VALIDATION)
predicted_validation_dataset = task.infer(
    validation_dataset.with_empty_annotations(),
    InferenceParameters(is_evaluation=True))

resultset = ResultSetEntity(
    model=output_model,
    ground_truth_dataset=validation_dataset,
    prediction_dataset=predicted_validation_dataset,
)
task.evaluate(resultset)
print(resultset.performance)


2022-01-26 13:55:01,455 | INFO : infer()
2022-01-26 13:55:01,458 | INFO : Confidence threshold 0.1
2022-01-26 13:55:01,459 | INFO : initializing....
2022-01-26 13:55:01,460 | INFO : called _init_recipe()
2022-01-26 13:55:01,462 | INFO : train type = SemiSupervised
2022-01-26 13:55:01,499 | INFO : initialized recipe = /mnt/hdd1/workspace/openvino/training_extensions/external/mpa/recipes/stages/detection/unbiased_teacher.yaml
2022-01-26 13:55:01,503 | WARNING : Duplicate key is detected among bases [{'model'}]
2022-01-26 13:55:01,507 | INFO : initialized.
2022-01-26 13:55:01,507 | INFO : running task... kwargs = {}
2022-01-26 13:55:01,507 | INFO : progress callback = None, hook name = ProgressUpdateHook
2022-01-26 13:55:01,507 | INFO : called build_recipe()
2022-01-26 13:55:01,508 | INFO : seems to be passed stage yaml...
2022-01-26 13:55:01,508 | INFO : build_stage()
2022-01-26 13:55:01,509 | INFO : work dir = /tmp/MPA-task-grphr6ux/stage00_DetectionInferrer-train
2022-01-26 13:55:01,50

## Export model to OpenVINO format

In [10]:
exported_model = ModelEntity(
    dataset,
    environment.get_model_configuration(),
    model_status=ModelStatus.NOT_READY)
task.export(ExportType.OPENVINO, exported_model)

2022-01-26 13:55:04,691 | INFO : Exporting the model
2022-01-26 13:55:04,693 | INFO : initializing....
2022-01-26 13:55:04,695 | INFO : called _init_recipe()
2022-01-26 13:55:04,696 | INFO : train type = SemiSupervised
2022-01-26 13:55:04,713 | INFO : initialized recipe = /mnt/hdd1/workspace/openvino/training_extensions/external/mpa/recipes/stages/detection/unbiased_teacher.yaml
2022-01-26 13:55:04,718 | WARNING : Duplicate key is detected among bases [{'model'}]
2022-01-26 13:55:04,721 | INFO : initialized.
2022-01-26 13:55:04,721 | INFO : running task... kwargs = {}
2022-01-26 13:55:04,721 | INFO : progress callback = None, hook name = ProgressUpdateHook
2022-01-26 13:55:04,721 | INFO : called build_recipe()
2022-01-26 13:55:04,722 | INFO : seems to be passed stage yaml...
2022-01-26 13:55:04,722 | INFO : build_stage()
2022-01-26 13:55:04,723 | INFO : work dir = /tmp/MPA-task-grphr6ux/stage00_DetectionExporter-train
2022-01-26 13:55:04,723 | INFO : CUDA_VISIBLE_DEVICES = None
2022-01

2022-01-26 13:55:04,972 - mmdet - INFO - load model from: torchvision://resnet50
2022-01-26 13:55:04,973 - mmdet - INFO - Use load_from_torchvision loader
2022-01-26 13:55:05,059 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias



2022-01-26 13:55:05,160 | INFO : ----------------- UnbiasedTeacher.state_dict_hook() called
2022-01-26 13:55:05,163 | INFO : ----------------- UnbiasedTeacher.state_dict_hook() called
2022-01-26 13:55:05,439 | INFO : ----------------- UnbiasedTeacher.state_dict_hook() called
2022-01-26 13:55:05,442 | INFO : ----------------- UnbiasedTeacher.state_dict_hook() called


/home/yunchu/workspace/openvino/training_extensions/mpa_env/lib/python3.8/site-packages/torch/onnx/symbolic_opset9.py:661: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn("This model contains a squeeze operation on dimension " + str(squeeze_dim) + ". If the model is " +


ONNX model has been saved to "/tmp/MPA-task-grphr6ux/stage00_DetectionExporter-train/export/model.onnx"
mo --input_model=/tmp/MPA-task-grphr6ux/stage00_DetectionExporter-train/export/model.onnx --mean_values=[123.675, 116.28, 103.53] --scale_values=[58.395, 57.12, 57.375] --output_dir=/tmp/MPA-task-grphr6ux/stage00_DetectionExporter-train/export --output=labels,boxes --data_type=FP32 --input_shape=[1, 3, 384, 384] --reverse_input_channels
Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/tmp/MPA-task-grphr6ux/stage00_DetectionExporter-train/export/model.onnx
	- Path for generated IR: 	/tmp/MPA-task-grphr6ux/stage00_DetectionExporter-train/export
	- IR output name: 	model
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	labels,boxes
	- Input shapes: 	[1, 3, 384, 384]
	- Mean values: 	[123.675, 116.28, 103.53]
	- Scale values: 	[58.395, 57.12, 57.375]
	- Scale fac

/mnt/hdd1/workspace/openvino/training_extensions/mpa_env/lib/python3.8/site-packages/mo/mo/utils/versions_checker.py:170: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  req_ver = LooseVersion(required_v)
/mnt/hdd1/workspace/openvino/training_extensions/mpa_env/lib/python3.8/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)


## Evaluate the exported model

In [ ]:
environment.model = exported_model
ov_task = get_impl_class(model_template.entrypoints.openvino)(environment)
predicted_validation_dataset = ov_task.infer(
    validation_dataset.with_empty_annotations(),
    InferenceParameters(is_evaluation=True))
resultset = ResultSetEntity(
    model=output_model,
    ground_truth_dataset=validation_dataset,
    prediction_dataset=predicted_validation_dataset,
)
ov_task.evaluate(resultset)
print(resultset.performance)

## Draw bounding boxes

In [ ]:
import IPython
import PIL
print(predicted_validation_dataset)
for predictions, item in zip(predicted_validation_dataset, validation_dataset.with_empty_annotations()):
    image = item.numpy.astype(np.uint8)
    print(f'annotation scene = {predictions.annotation_scene}')
    for box in predictions.annotation_scene.shapes:
        x1 = int(box.x1 * image.shape[1])
        x2 = int(box.x2 * image.shape[1])
        y1 = int(box.y1 * image.shape[0])
        y2 = int(box.y2 * image.shape[0])
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 3)  
        print(f'bbox {x1,y1}-{x2,y2}')
    # IPython.display.display(PIL.Image.fromarray(image))
    display(PIL.Image.fromarray(image))